In [98]:
#!pip3 install tensorflow
!pip3 install keras-tuner --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [108]:
!pip3 uninstall tensorflow

^C


In [110]:
!pip3 uninstall keras

^C


In [115]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import BinaryAccuracy, Recall, Precision, AUC

In [60]:
!pip3 install tf-nightly

Defaulting to user installation because normal site-packages is not writeable
  Using cached tf_nightly-2.14.0.dev20230620-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached tf_nightly_intel-2.14.0.dev20230620-cp39-cp39-win_amd64.whl (183.9 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Anthony\\AppData\\Roaming\\Python\\Python39\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Check the permissions.



In [31]:
df = pd.read_csv('Phishing_Mitre_Dataset_Summer_of_AI.csv')
df.head()

,create_age(months),expiry_age(months),update_age(days),URL,Label
0,-1,-1,-1,http://account-google-com.ngate.my/c44cca40176...,1
1,212,16,663,http://www.coffeespecialties.com/...,0
2,-1,-1,-1,http://black.pk/wp-content/2013/04/bp.postale/...,1
3,198,6,186,http://atomicsoda.com/manutd...,0
4,240,24,1684,http://bostoncoffeecake.com/...,0


In [38]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=80)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['URL'])

In [39]:
train_sequences = tokenizer.texts_to_sequences(train_df['URL'])
test_sequences = tokenizer.texts_to_sequences(test_df['URL'])

In [40]:
max_sequence_length = max(len(seq) for seq in train_sequences)
train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

In [41]:
train_labels = train_df['Label'].values
test_labels = test_df['Label'].values

In [124]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01, 0.1],
    'lstm_units': [16, 32, 64, 128, 256],
    'optimizer': ['adam', 'rmsprop'],
}

In [122]:
# Create the RNN model
def create_model(optimizer, dropout_rate=0.2, learning_rate=0.001, lstm_units=64):
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_sequence_length))
    model.add(LSTM(lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall', 'AUC'])

    return model


In [125]:
# Compile the model
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# Create GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=3)

grid_search.fit(train_data, train_labels)

C:\Users\Anthony\AppData\Local\Temp\ipykernel_16032\468974290.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
70/70 [==============================] - 3s 11ms/step - loss: 0.5575 - accuracy: 0.7119
Epoch 2/10
70/70 [==============================] - 1s 11ms/step - loss: 0.2391 - accuracy: 0.9254
Epoch 3/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0843 - accuracy: 0.9821
Epoch 4/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0340 - accuracy: 0.9969
Epoch 5/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0202 - accuracy: 0.9982
Epoch 6/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0117 - accuracy: 0.9996
Epoch 7/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0083 - accuracy: 0.9996
Epoch 8/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 9/10
70/70 [==============================] - 1s 11ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 10/10
35/35 [==============================] - 1s 3ms/step - loss: 0.1980 - accuracy: 0.9411
Epoch 1/10

KeyboardInterrupt: 

In [ ]:
# Get the best model
best_model = grid_search.best_estimator_

In [89]:
# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=32)

Epoch 1/10
105/105 [==============================] - 3s 13ms/step - loss: 0.3984 - binary_accuracy: 0.8327 - precision_3: 0.8911 - recall_3: 0.7787 - auc_2: 0.9140
Epoch 2/10
105/105 [==============================] - 1s 14ms/step - loss: 0.0878 - binary_accuracy: 0.9702 - precision_3: 0.9767 - recall_3: 0.9668 - auc_2: 0.9958
Epoch 3/10
105/105 [==============================] - 1s 13ms/step - loss: 0.0155 - binary_accuracy: 0.9958 - precision_3: 0.9983 - recall_3: 0.9938 - auc_2: 0.9999
Epoch 4/10
105/105 [==============================] - 1s 13ms/step - loss: 0.0049 - binary_accuracy: 0.9991 - precision_3: 1.0000 - recall_3: 0.9983 - auc_2: 1.0000
Epoch 5/10
105/105 [==============================] - 1s 13ms/step - loss: 0.0018 - binary_accuracy: 1.0000 - precision_3: 1.0000 - recall_3: 1.0000 - auc_2: 1.0000
Epoch 6/10
105/105 [==============================] - 1s 13ms/step - loss: 6.7447e-04 - binary_accuracy: 1.0000 - precision_3: 1.0000 - recall_3: 1.0000 - auc_2: 1.0000
Epoch 

In [90]:
loss, accuracy, precision, recall, auc = model.evaluate(test_data, test_labels)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test precision: {precision:.4f}')
print(f'Test recall: {recall:.4f}')
print(f'Test AUC: {auc:.4f}')

45/45 [==============================] - 1s 5ms/step - loss: 0.2529 - binary_accuracy: 0.9465 - precision_3: 0.9848 - recall_3: 0.9153 - auc_2: 0.9751
Test loss: 0.2529
Test accuracy: 0.9465
Test precision: 0.9848
Test recall: 0.9153
Test AUC: 0.9751
